In [1]:
!pip install -q predibase datasets

& was unexpected at this time.
The value specified in an AutoRun registry key could not be parsed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.37.2 requires urllib3~=2.0, but you have urllib3 1.26.12 which is incompatible.


In [1]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
from datasets import load_dataset
import csv

def hfdataset_to_csv(datalist: list, csv_file_name, max=-1):
  template = {
        "prompt":
        """<|im_start|>system\nYou are an expert classifer. Based on the given Sentece, Classfity them as one of the four category:korean general law, korean business law, japanese general law,japanese business law  <|im_end|>
<|im_start|>Input\n {content}
<|im_start|>Class\n""",
        "completion": "{headline}<|im_end|>",
        "split": "train"}


  with open(csv_file_name, 'w', newline='') as csvfile:
    fieldnames = template.keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, d in enumerate(datalist):
      if i >= max:
        break
      row = {
          "prompt": template["prompt"].format(content=d["question"]),
          "completion": template["completion"].format(headline=d["adapter_name"]),
          "split": "train"
      }
      writer.writerow(row)

In [4]:
def validate_data_csv(csv_file_name):
  """ Make sure it has prompt, completion, and split with all values """
  with open(csv_file_name, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
      assert row['prompt']
      assert row['completion']
      assert row['split']

  return True

In [5]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

def compute_cost(csv_file_name, price_per_million_tokens=0.5):
  """ Compute the cost of the dataset """

  total_num_of_tokens = 0
  with open(csv_file_name, 'r') as f:
    reader = csv.DictReader(f)
    # get all values
    values = [row['completion']+ " " + row['prompt'] for row in reader]
    for value in values:
      # tokenize
      enc = tokenizer.encode(value)
      num_of_tokens = len(enc.tokens)
      total_num_of_tokens += num_of_tokens


  return total_num_of_tokens / 1000000 * price_per_million_tokens

In [6]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

# Load the dataset
hfdataset = load_dataset("csv", data_files="adapter_classification.csv")
train_hfdataset = hfdataset["train"]
print(train_hfdataset)
# Convert the dataset to a pandas DataFrame for easier manipulation
train_df = train_hfdataset.to_pandas()

# Split the train dataset into 80% train and 20% test
train_split, test_split = train_test_split(train_df, test_size=0.1, random_state=42)

train_split.reset_index(drop=True, inplace=True)
test_split.reset_index(drop=True, inplace=True)

# Convert back to the Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_split)
test_dataset = Dataset.from_pandas(test_split)

# Print the sizes of the new datasets
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(train_dataset)

Dataset({
    features: ['question', 'adapter_name'],
    num_rows: 2000
})
Train dataset size: 1800
Test dataset size: 200
Dataset({
    features: ['question', 'adapter_name'],
    num_rows: 1800
})


In [12]:
dataset_name = "adapter_train_dataset_1800"
csv_file_name = f"{dataset_name}.csv"
try:
  pb_dataset = pb.datasets.get(dataset_name)
  print(f"Dataset found: {pb_dataset}")
except RuntimeError:
  print("Dataset not found, creating...")

  hfdataset_to_csv(train_dataset, csv_file_name, max=1800)

  print(f"Dataset Validation: {validate_data_csv(csv_file_name)}")
  print(f"One step FT Cost: {compute_cost(csv_file_name)} USD")

  #print("Uploading daatset...")
  pb_dataset = pb.datasets.from_file(csv_file_name, name=dataset_name)

Dataset found: uuid='5bffbbac-3a56-49b9-a197-bee620959817' name='adapter_train_dataset_1800' connection_type='file' connection_name='file_uploads' status='connected'


In [8]:
repo_name = "adapter-classifer-01"
repo = pb.repos.create(name=repo_name, description="Upstage_Hackathon_KGL_Adapter", exists_ok=True)
print(repo)

uuid='183e5ef8-124b-4b00-9fe5-9f27266b0484' name='adapter-classifer-01' description='Upstage_Hackathon_KGL_Adapter'


In [13]:
adapter = pb.adapters.create(
    config=FinetuningConfig(
        base_model="solar-1-mini-chat-240612",
        epochs=3, # default: 3
        rank=16, # default: 16
    ),
    dataset=pb_dataset, # Also accepts the dataset name as a string
    repo=repo,
    description="initial model with defaults"
)

Successfully requested finetuning of solar-1-mini-chat-240612 as `adapter-classifer-01/2`. (Job UUID: 0245a50d-1bfc-4fca-8c5f-c360e67d2fd3).

Watching progress of finetuning job 0245a50d-1bfc-4fca-8c5f-c360e67d2fd3. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:00:47         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   0.0000   │        --       │
└────────────┴────────────┴─────────────────┘


In [14]:
adapter

Adapter(repo='adapter-classifer-01', tag=2, archived=False, base_model='solar-1-mini-chat-240612', description='initial model with defaults', artifact_path='0245a50d-1bfc-4fca-8c5f-c360e67d2fd3/52546414125f4180872c76b3bf450da6/artifacts/model/model_weights', finetuning_error=None, finetuning_job_uuid='0245a50d-1bfc-4fca-8c5f-c360e67d2fd3')

In [15]:
adapter_id = adapter.repo + "/" + str(adapter.tag)
adapter_id

'adapter-classifer-01/2'

In [16]:
input_prompt="""
<|im_start|>system\nYou are an expert classifer. Based on the given Sentece, Classfity them as one of the four category:korean general law, korean business law, japanese general law,japanese business law<|im_end|>
<|im_start|>Input
피고인은 언제, 어디서, 어떤 차량을 운전했습니까? <|im_end|>
<|im_start|>Class 
"""

In [17]:
# CURL test
import requests
import json

url = "https://serving.app.predibase.com/7ea6d0/deployments/v2/llms/solar-1-mini-chat-240612/generate"


payload = {
    "inputs": input_prompt,
    "parameters": {
        "adapter_id": adapter_id,
        "adapter_source": "pbase",
        "max_new_tokens": 60,
        "temperature": 0.1
    }
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_token}"
}

response = requests.post(url, data=json.dumps(payload), headers=headers)

print(response.text)

{"generated_text":"korean general law"}


In [15]:
!pip -q install langchain langchain-upstage solar-as-judge

& was unexpected at this time.
The value specified in an AutoRun registry key could not be parsed.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.37.2 requires urllib3~=2.0, but you have urllib3 1.26.12 which is incompatible.


In [19]:
lorax_client = pb.deployments.client("solar-1-mini-chat-240612")

In [21]:
# Create test dataset
test_csv_file_name = f"adapter_test_dataset"
hfdataset_to_csv(test_dataset, test_csv_file_name, max=200)

win_results = {"A_wins":0, "B_wins": 0, "tie": 0,  "A_score": 0, "B_score": 0}
with open(test_csv_file_name, 'r') as f:
  reader = csv.DictReader(f)
  for row in reader:
    prompt = row['prompt']
    ground_truth = row['completion']
    A_answer = lorax_client.generate(prompt, max_new_tokens=1000).generated_text
    B_answer = lorax_client.generate(prompt, adapter_id=adapter_id, max_new_tokens=1000).generated_text

    A_score, B_score = saj.judge(prompt, A_answer, B_answer, ground_truth)
    print( A_score, B_score, A_answer, B_answer)
    win_results["A_score"] += A_score
    win_results["B_score"] += B_score
    if A_score > B_score:
      win_results["A_wins"] += 1
    elif B_score > A_score:
      win_results["B_wins"] += 1
    else:
      win_results["tie"] += 1
    print(win_results)


2 10 korean business law japanese business law
{'A_wins': 0, 'B_wins': 1, 'tie': 0, 'A_score': 2, 'B_score': 10}
0 0 korean general law korean general law
{'A_wins': 0, 'B_wins': 1, 'tie': 1, 'A_score': 2, 'B_score': 10}
0 0 일본 일반법 japanese general law
{'A_wins': 0, 'B_wins': 1, 'tie': 2, 'A_score': 2, 'B_score': 10}
0 0 korean business law korean business law
{'A_wins': 0, 'B_wins': 1, 'tie': 3, 'A_score': 2, 'B_score': 10}
Invalid json output: The provided answer is: japanese general law

Based on the prompt and the ground truth, I evaluate the answer as follows:

{"score": 5}
0 0 日本の一般法 japanese general law
{'A_wins': 0, 'B_wins': 1, 'tie': 4, 'A_score': 2, 'B_score': 10}
0 0 日本の一般法 japanese general law
{'A_wins': 0, 'B_wins': 1, 'tie': 5, 'A_score': 2, 'B_score': 10}
0 0 korean business law korean business law
{'A_wins': 0, 'B_wins': 1, 'tie': 6, 'A_score': 2, 'B_score': 10}
2 10 korean business law japanese business law
{'A_wins': 0, 'B_wins': 2, 'tie': 6, 'A_score': 4, 'B_score':

In [22]:
print(win_results)

{'A_wins': 0, 'B_wins': 60, 'tie': 140, 'A_score': 132, 'B_score': 587}
